In [7]:
### 1번 블럭: 기본 설정 및 데이터 준비

import pandas as pd
import numpy as np
import os
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# 데이터 읽기
base_path = '/Users/foxrainswap/Desktop/데이터/재차인원/5000B/합본'
train_data = pd.read_csv(os.path.join(base_path, '트레이닝셋_정규화x_이상치제거.csv'))
test_data = pd.read_csv(os.path.join(base_path, '테스트셋_정규화x.csv'))

# 날짜 변환
train_data['날짜'] = pd.to_datetime(train_data['날짜'])
test_data['날짜'] = pd.to_datetime(test_data['날짜'])

def prepare_data(data):
    """데이터 전처리 - Prophet 형식에 맞게 변환"""
    time_columns = ['04시', '05시', '06시', '07시', '08시', '09시', '10시', 
                   '11시', '12시', '13시', '14시', '15시', '16시', '17시', 
                   '18시', '19시', '20시', '21시', '22시', '23시', '00시', 
                   '01시', '02시', '03시']
    
    melted_data = pd.melt(data, 
                         id_vars=['날짜', '정류장순번', '정류장명', '요일'], 
                         value_vars=time_columns,
                         var_name='시간', 
                         value_name='y')  # Prophet은 타겟변수를 'y'로 사용
    
    # datetime 컬럼을 'ds' 로 변환 (Prophet 요구사항)
    melted_data['ds'] = pd.to_datetime(melted_data['날짜'].astype(str) + ' ' + 
                                     melted_data['시간'].str.replace('시', ':00'))
    
    # 평일/토요일/일요일 구분
    melted_data['요일구분'] = melted_data['요일'].map({
        '토요일': '토요일',
        '일요일': '일요일'
    }).fillna('평일')
    
    melted_data.sort_values(['정류장순번', 'ds'], inplace=True)
    
    return melted_data

def post_process_predictions(predictions):
    """예측값 후처리"""
    processed = predictions.copy()
    processed[processed < 1] = 0  # 1미만 값을 0으로
    processed[processed < 0] = 0  # 음수 값을 0으로
    return processed

def calculate_evaluation_metrics(y_true, y_pred):
    """평가 지표 계산"""
    # 예측값 후처리
    processed_pred = post_process_predictions(y_pred)
    
    # 기본 지표 계산
    mae = mean_absolute_error(y_true, processed_pred)
    rmse = np.sqrt(mean_squared_error(y_true, processed_pred))
    
    # SMAPE 계산
    numerator = np.abs(y_true - processed_pred)
    denominator = np.abs(y_true) + np.abs(processed_pred)
    
    zero_mask = denominator == 0
    smape = np.mean(
        np.where(zero_mask, 0, numerator / denominator)
    ) * 200
    
    return {
        'MAE': mae,
        'RMSE': rmse,
        'SMAPE': smape
    }

In [8]:
### 2번 블럭: Prophet 분석 함수

def analyze_all_stations_prophet(train_data, test_data):
    """모든 정류장에 대한 Prophet 분석 수행"""
    time_columns = ['04시', '05시', '06시', '07시', '08시', '09시', '10시', 
                   '11시', '12시', '13시', '14시', '15시', '16시', '17시', 
                   '18시', '19시', '20시', '21시', '22시', '23시', '00시', 
                   '01시', '02시', '03시']
    
    # 운행 시간대 정의
    operating_hours = ['12시', '13시', '14시', '15시', '16시', '17시', 
                      '18시', '19시', '20시', '21시', '22시', '23시', 
                      '00시', '01시']
    
    # 첨두시간대 정의
    peak_hours = ['17시', '18시', '19시', '22시', '23시']
    
    prepared_train = prepare_data(train_data)
    prepared_test = prepare_data(test_data)
    
    weekday_results = []
    saturday_results = []
    sunday_results = []
    evaluation_results = []
    
    all_stations = train_data['정류장순번'].unique()
    total_stations = len(all_stations)

    for idx, station_id in enumerate(all_stations, 1):
        print(f"\n정류장 처리 중... ({idx}/{total_stations})")
        station_name = train_data[train_data['정류장순번'] == station_id]['정류장명'].iloc[0]
        
        for day_type in ['평일', '토요일', '일요일']:
            try:
                # 데이터 준비
                train_station = prepared_train[
                    (prepared_train['요일구분'] == day_type) & 
                    (prepared_train['정류장순번'] == station_id)
                ].sort_values('ds').reset_index(drop=True)
                
                test_station = prepared_test[
                    (prepared_test['요일구분'] == day_type) & 
                    (prepared_test['정류장순번'] == station_id)
                ].sort_values('ds').reset_index(drop=True)
                
                # Prophet 모델 학습
                model = Prophet(
                    yearly_seasonality=False,
                    weekly_seasonality=False,
                    daily_seasonality=True,
                    changepoint_prior_scale=0.05
                )
                
                model.fit(train_station[['ds', 'y']])
                
                # 예측 수행
                future = test_station[['ds']].copy()
                forecast = model.predict(future)
                predictions = forecast['yhat'].values
                
                # 시간대별 평균 예측값 계산
                predictions_by_hour = {}
                test_station['시간'] = test_station['ds'].dt.strftime('%H시')
                unique_times = test_station['시간'].unique()
                pred_splits = np.array_split(predictions, len(unique_times))
                
                for time, preds in zip(unique_times, pred_splits):
                    predictions_by_hour[time] = np.mean(preds) if time in operating_hours else 0
                
                # 결과 저장
                result_dict = {
                    '정류장순번': station_id,
                    '정류장명': station_name
                }
                for time in time_columns:
                    result_dict[time] = predictions_by_hour.get(time, 0)
                
                # 요일별 결과 저장
                if day_type == '평일':
                    weekday_results.append(result_dict)
                elif day_type == '토요일':
                    saturday_results.append(result_dict)
                else:  # 일요일
                    sunday_results.append(result_dict)
                
                # 평가 수행
                # 전체 운행시간대 평가
                test_station_operating = test_station[test_station['시간'].isin(operating_hours)].copy()
                operating_predictions = np.array([pred for i, pred in enumerate(predictions) 
                                               if test_station.iloc[i]['시간'] in operating_hours])
                
                # 첨두시간대 평가
                test_station_peak = test_station[test_station['시간'].isin(peak_hours)].copy()
                peak_predictions = np.array([pred for i, pred in enumerate(predictions) 
                                          if test_station.iloc[i]['시간'] in peak_hours])
                
                # 평가 지표 계산
                operating_metrics = calculate_evaluation_metrics(
                    test_station_operating['y'].values,
                    operating_predictions
                )
                
                peak_metrics = calculate_evaluation_metrics(
                    test_station_peak['y'].values,
                    peak_predictions
                )
                
                evaluation_results.append({
                    '정류장순번': station_id,
                    '정류장명': station_name,
                    '요일구분': day_type,
                    '구분': '전체',
                    **operating_metrics
                })
                
                evaluation_results.append({
                    '정류장순번': station_id,
                    '정류장명': station_name,
                    '요일구분': day_type,
                    '구분': '첨두',
                    **peak_metrics
                })
                
            except Exception as e:
                print(f"오류 발생 - 정류장: {station_id}, 구분: {day_type}")
                print(f"오류 내용: {str(e)}")
                continue
    
    # 결과를 DataFrame으로 변환
    weekday_df = pd.DataFrame(weekday_results)
    saturday_df = pd.DataFrame(saturday_results)
    sunday_df = pd.DataFrame(sunday_results)
    evaluation_df = pd.DataFrame(evaluation_results)
    
    # 결과 저장
    weekday_df.to_csv('prophet_predictions_weekday.csv', index=False)
    saturday_df.to_csv('prophet_predictions_saturday.csv', index=False)
    sunday_df.to_csv('prophet_predictions_sunday.csv', index=False)
    evaluation_df.to_csv('prophet_evaluation.csv', index=False)
    
    return weekday_df, saturday_df, sunday_df, evaluation_df

In [9]:
### 3번 블럭: 실행 코드

# Prophet 분석 실행
print("\nProphet 모델 실행 중...")
weekday_df, saturday_df, sunday_df, evaluation_df = analyze_all_stations_prophet(train_data, test_data)

print("\n분석 완료!")
print("결과 파일이 저장되었습니다:")
print("- prophet_predictions_weekday.csv")
print("- prophet_predictions_saturday.csv")
print("- prophet_predictions_sunday.csv")
print("- prophet_evaluation.csv")


Prophet 모델 실행 중...

정류장 처리 중... (1/79)


21:29:29 - cmdstanpy - INFO - Chain [1] start processing
21:29:31 - cmdstanpy - INFO - Chain [1] done processing
21:29:31 - cmdstanpy - INFO - Chain [1] start processing
21:29:31 - cmdstanpy - INFO - Chain [1] done processing
21:29:32 - cmdstanpy - INFO - Chain [1] start processing
21:29:32 - cmdstanpy - INFO - Chain [1] done processing
21:29:32 - cmdstanpy - INFO - Chain [1] start processing



정류장 처리 중... (2/79)


21:29:32 - cmdstanpy - INFO - Chain [1] done processing
21:29:33 - cmdstanpy - INFO - Chain [1] start processing
21:29:33 - cmdstanpy - INFO - Chain [1] done processing
21:29:33 - cmdstanpy - INFO - Chain [1] start processing
21:29:33 - cmdstanpy - INFO - Chain [1] done processing
21:29:33 - cmdstanpy - INFO - Chain [1] start processing



정류장 처리 중... (3/79)


...


정류장 처리 중... (78/79)


21:31:00 - cmdstanpy - INFO - Chain [1] start processing
21:31:00 - cmdstanpy - INFO - Chain [1] done processing
21:31:00 - cmdstanpy - INFO - Chain [1] start processing
21:31:00 - cmdstanpy - INFO - Chain [1] done processing
21:31:00 - cmdstanpy - INFO - Chain [1] start processing



정류장 처리 중... (79/79)


21:31:00 - cmdstanpy - INFO - Chain [1] done processing
21:31:01 - cmdstanpy - INFO - Chain [1] start processing
21:31:01 - cmdstanpy - INFO - Chain [1] done processing
21:31:01 - cmdstanpy - INFO - Chain [1] start processing
21:31:01 - cmdstanpy - INFO - Chain [1] done processing



분석 완료!
결과 파일이 저장되었습니다:
- prophet_predictions_weekday.csv
- prophet_predictions_saturday.csv
- prophet_predictions_sunday.csv
- prophet_evaluation.csv
